<a href="https://colab.research.google.com/github/LeaCarop/Conv_Net/blob/main/Parte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**## Parte 3: Clasificación de imágenes de gatos vs perros**

En este ejercicio, veremos dos técnicas para reutilizar datos de características generados a partir de modelos de imágenes que ya han sido entrenados en grandes conjuntos de datos, extracción de características y ajuste fino, y los usaremos para mejorar la precisión del modelo de clasificación de gatos y perros. Ya que, en la parte 3 se aumentó el accuracy entorno al 75%, pero sigue habiendo un 25% de error.

Voy a utilizar el modelo Inception V3 desarrollado en Google y entrenado previamente en ImageNet, un gran conjunto de datos de imágenes web (1,4 millones de imágenes y 1000 clases). 

Primero, debemos elegiré qué capa intermedia de Inception V3 usaré para la extracción de características. Una práctica común es usar la salida de la última capa antes de la operación Flatten, la llamada "capa de cuello de botella". El razonamiento aquí es que las siguientes capas completamente conectadas serán demasiado especializadas para la tarea en la que se entrenó la red y, por lo tanto, las funciones aprendidas por estas capas no serán muy útiles para una nueva tarea. Las características de cuello de botella, sin embargo, conservan mucha generalidad.

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-03-18 13:13:02--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  44.9MB/s    in 1.9s    

2022-03-18 13:13:04 (44.9 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False


La capa que usaré para la extracción de características en la Inception V3 se llama mixed7. No es el cuello de botella de la red, pero lo estamos usando para mantener un mapa de características suficientemente grande (7x7 en este caso). (El uso de la capa de cuello de botella habría dado como resultado un mapa de características de 3x3, que es un poco pequeño). Obtengamos el resultado de mixed7:

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)

x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation='sigmoid')(x)


model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
!wget --no-check-certificate \
   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O \
   /tmp/cats_and_dogs_filtered.zip

--2022-03-18 13:13:28--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 172.217.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   146MB/s    in 0.4s    

2022-03-18 13:13:28 (146 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')

train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')

validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Añadir parámetros de data-augmentation a ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Los datos de validación no deben ser aumentados!!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),  
        batch_size=20,
      
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# ENTRENAMIENTO
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/2
100/100 - 117s - loss: 0.3534 - acc: 0.8555 - val_loss: 0.1621 - val_acc: 0.9340 - 117s/epoch - 1s/step
Epoch 2/2
100/100 - 113s - loss: 0.2337 - acc: 0.9110 - val_loss: 0.1002 - val_acc: 0.9650 - 113s/epoch - 1s/step


Hemos mejorado el modelo e comparación a la Parte 2 (Accuracy-0.75%). Ahora tenemos una precisión en validación de 0.91%

### **Mejorando aún más la precisión con Fine-Turing**

En nuestro experimento de extracción de características, solo intentamos agregar dos capas de clasificación encima de una capa de Inception V3. Los pesos de la red preentrenada no se actualizaron durante el entrenamiento. Una forma de aumentar aún más el rendimiento es "afinar" los pesos de las capas superiores del modelo preentrenado junto con el entrenamiento del clasificador de nivel superior. Un par de notas importantes sobre Fine-Turing:

- Fine-Turing solo debe intentarse después de haber entrenado el clasificador de nivel superior con el modelo preentrenado establecido en no entrenable. Si se agrega un clasificador inicializado aleatoriamente encima de un modelo preentrenado e intentas entrenar todas las capas juntas, la magnitud de las actualizaciones de gradiente serán demasiado grandes (debido a los pesos aleatorios del clasificador) y tu modelo preentrenado simplemente olvidará todo lo que ha aprendido
- Además, ajustamos con precisión solo las capas superiores del modelo preentrenado en lugar de todas las capas del modelo preentrenado porque, en una convnet, cuanto más arriba está una capa, más especializada es. Las primeras capas en una convnet aprenden características muy simples y genéricas, que se generalizan a casi todos los tipos de imágenes. Pero a medida que avanza, las funciones son cada vez más específicas para el conjunto de datos en el que se entrena el modelo. El objetivo del Fine-Turing es adaptar estas características especializadas para que funcionen con el nuevo conjunto de datos.

Todo lo que hay que hacer para implementar el Fine-Turing es configurar las capas superiores de Inception V3 para que se puedan entrenar, volver a compilar el modelo (necesario para que estos cambios surtan efecto) y reanudar el entrenamiento. Descongelar todas las capas que pertenecen al módulo mixed7, es decir, todas las capas encontradas después de mixed6, y volver a compilar el modelo:

In [ ]:
from tensorflow.keras.optimizers import SGD

unfreeze = False

# Unfreeze todos los modelos despues de "mixed6"
for layer in pre_trained_model.layers:
  if unfreeze:
    layer.trainable = True
  if layer.name == 'mixed6':
    unfreeze = True

model.compile(loss='binary_crossentropy',
              optimizer=SGD(
                  lr=0.00001, # ratio de aprendizaje muy bajo
                  momentum=0.9),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
# al tener 50 épocas va a demorar en entrenar
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/50
100/100 - 157s - loss: 0.3165 - acc: 0.8600 - val_loss: 0.1124 - val_acc: 0.9500 - 157s/epoch - 2s/step
Epoch 2/50
100/100 - 152s - loss: 0.2690 - acc: 0.8845 - val_loss: 0.1142 - val_acc: 0.9560 - 152s/epoch - 2s/step
Epoch 3/50
100/100 - 152s - loss: 0.2277 - acc: 0.9030 - val_loss: 0.1169 - val_acc: 0.9540 - 152s/epoch - 2s/step
Epoch 4/50
100/100 - 152s - loss: 0.2264 - acc: 0.9095 - val_loss: 0.1190 - val_acc: 0.9560 - 152s/epoch - 2s/step
Epoch 5/50
100/100 - 150s - loss: 0.2261 - acc: 0.9050 - val_loss: 0.1201 - val_acc: 0.9530 - 150s/epoch - 1s/step
Epoch 6/50
100/100 - 150s - loss: 0.2030 - acc: 0.9140 - val_loss: 0.1213 - val_acc: 0.9510 - 150s/epoch - 1s/step
Epoch 7/50
100/100 - 150s - loss: 0.2139 - acc: 0.9150 - val_loss: 0.1215 - val_acc: 0.9510 - 150s/epoch - 1s/step
Epoch 8/50
100/100 - 150s - loss: 0.2260 - acc: 0.8985 - val_loss: 0.1205 - val_acc: 0.9520 - 150s/epoch - 1s/step
Epoch 9/50
100/100 - 150s - loss: 0.1967 - acc: 0.9235 - val_loss: 0.1206 - val_

He interrumpido el entrenamiento porque me lleva más de media hora, pero haber entrenado 40 épocas me permite saber que he mejorado un poco mi modelo, ahora tendrá un precisión que ronda el 92%, no es mucho más, pero es una leve mejora.

Voy a trazar la pérdida y la precisión del entrenamiento y la validación para demostrarlo de manera concluyente:
(Realmente dejo el código indicado de la representación, que sería igual a lo hecho en las partes 1 y 2, pero al no haber entrenado el modelo anterior al completo no puedo representarlo graficamente).

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']


plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')